# 봄이 왔어요^^

# 크롤링

In [2]:
import requests # http요청할 때 사용
from bs4 import BeautifulSoup # html정보를 간단하게 가져올 때

In [3]:
# 가져오고싶은 정보가 있는 url
base_url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=184517&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=sympathyScore&page={}'
url = base_url.format(1) # url이라는 변수에 저장(1 : 페이지 번호)
res=requests.get(url) # 요청하여 가져오기

In [13]:
# 좀 많긴해도 content내용에 어떤 데이터가 있는지 확인해보자
html = BeautifulSoup(res.content, 'html.parser')
html


<!DOCTYPE html>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>네이버 영화</title>
<link href="https://ssl.pstatic.net/static/m/movie/icons/naver_movie_favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/css/common.css?20210514151210" rel="stylesheet" type="text/css">
<link href="/css/movie_tablet.css?20210514151210" rel="stylesheet" type="text/css"/>
<link href="/css/movie_end.css?20210514151210" rel="stylesheet" type="text/css"/>
<script src="/js/deploy/movie.all.js?20210514151210" type="text/javascript"></script>
</link></head>
<body>
<!-- content -->
<input id="movieCode" name="movieCode" type="hidden" value="184517"/>
<input id="onlyActualPointYn" name="onlyActualPointYn" type="hidden" value="N"/>
<input id="includeSpoilerYn" name="includeSpoilerYn" type="hidden" value="N"/>
<input id="order" name="order" type="hidden" value="sympathyScore"/>
<input id="page" name="page" type="hidden" value="1"/>
<div clas

가져오고 싶은 데이터의 위치를 파악해야한다.<br>
크롬으로 url을 열고 f12키를 눌러 개발자모드를 활성화 한 후 *ctrl + shift + c* 키를 누르면 원하는 화면을 클릭할 수 있는데<br>
이 때 Elements안에있는 html문에서 누른 화면의 코드위치를 나타내준다.<br>
<br><br>
네이버 리뷰의 경우 댓글은 \<div class='score_result'>에 속해있는 것을 확인할 수 있다.<br>
div에는 ui가 있고 ui안에 li가 있는데 li안에 하나의 댓글의 정보가 담겨져 있기에 li가 10번 반복되고 있다.( div > ui > li )<br>

In [16]:
# 이제 양파까듯이 하나 하나 까보도록 하자
scoer_result = html.find('div',{'class':'score_result'})
lis = scoer_result.findAll('li')

In [18]:
lis[0] # 첫 번째 댓글에 담긴 내용

<li>
<div class="star_score">
<span class="st_off"><span class="st_on" style="width:100.0%"></span></span><em>10</em>
</div>
<div class="score_reple">
<p>
<span class="ico_viewer">관람객</span>
<!-- 스포일러 컨텐츠로 처리되는지 여부 -->
<span id="_filtered_ment_0">
														
															
															
																당신이 무의미하다고 느끼는 일상의 나열같은 인생도 무의미하지 않다는걸 보여주는 연출에서 강한 인상을 받았습니다 눈에 보이지 않는 "삶의 목적"만 찾다가 현재를 낭비하지말고 살아간다는 자체를 즐기세요! 
															
														
														
													</span>
</p>
<dl>
<dt>
<em>
<a href="#" onclick="javascript:showPointListByNid(17344608, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>SIN(ultr****)</span>
</a>
</em>
<em>2021.01.20 13:41</em>
</dt>
<dd>
<a class="go_report2" href="#" onclick="parent.clickcr(this, 'ara.report', '', '', event); common.report('false','ultr****', 'nKfYwEGDDL5RBjoQLZDlJySt6coDXUIVVisn/DM/vDU=', '당신이 무의미하다고 느끼는 일상의 나열같은 인생도 무의미하지 않다는걸 보여주는 연출에서 강한 인상을 받았습니다 눈에 보이지 않는 &quot;삶의 목적&quot;만 찾

## 댓글 내용

In [61]:
review_text = lis[1].find('p').getText()

if '관람객' in review_text:
    spectator = 1 # 관람객인경우
else:
    spectator = 0 # 관람객이 아닐경우

review_text = review_text.replace('관람객','')
review_text = review_text.replace('\n','') # 줄바꿈표시 제거
review_text = review_text.replace('\t','') # 탭 제거
review_text = review_text.replace('\r','') # 맨앞표시 제거
review_text

'\n관람객\n\n\n\n\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t인생은 목적이 있어서, 이유가 있어서 사는게 아니라 사는 그 자체가 인생의 목적과 이유가 될수 있다는게 마음이 어딘가 슬프고 벅찬 느낌이 들었다. 나는 왜 태어났지, 내 인생의 불꽃은 뭔지 그걸 꼭 알아내고 찾아야만...\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\n\n'

## 평점

In [20]:
score = lis[0].find('em').getText()
score

'10'

## 댓글 좋아요/싫어요 수

In [31]:
like = lis[0].find('div', {'class': 'btn_area'}).findAll('strong')[0].getText()
dislike = lis[0].find('div', {'class': 'btn_area'}).findAll('strong')[1].getText()
like, dislike

('3472', '82')

## 닉네임

In [84]:
nickname = lis[0].findAll('a')[0].find('span').getText()
nickname

'SIN(ultr****)'

## 작성일

In [39]:
from datetime import datetime

In [47]:
lis[0].find('dt').findAll('em')[-1].getText()

'2021.01.20 13:41'

In [45]:
date = datetime.strptime(lis[0].find('dt').findAll('em')[-1].getText(), "%Y.%m.%d %H:%M")
date

datetime.datetime(2021, 1, 20, 13, 41)

In [50]:
print(date.year, date.month, date.day)

2021 1 20


## 전체 댓글 수

In [66]:
result = html.find('div', {'class':'score_total'}).find('strong').findChildren('em')[0].getText()
int(result.replace(',', ''))

9735

# 판다스로 데이터 저장하기

In [69]:
import pandas as pd
from tqdm import tqdm

In [75]:
base_url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=184517&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=sympathyScore&page={}'

result = html.find('div', {'class':'score_total'}).find('strong').findChildren('em')[0].getText()
total_count = int(result.replace(',', ''))

In [76]:
total_count

9735

In [134]:
nicknames, scores, spectators, review_texts, likes, dislikes, date_list = [], [], [], [], [], [], []
for i in tqdm(range(int(total_count / 10)+1)): # 페이지 수만큼 반복
    url = base_url.format(i+1) # url이라는 변수에 저장(1 : 페이지 번호)
    res=requests.get(url) # 요청하여 가져오기
    html = BeautifulSoup(res.content, 'html.parser') # 파싱
    scoer_result = html.find('div',{'class':'score_result'}) 
    lis = scoer_result.findAll('li') # 댓글정보 가져오기(10개씩)
    for j in range(len(lis)):
        # 닉네임 - 닉네임이 저장된 'a'의 위치가 댓글별로 상이하기에 댓글이 존재하는 위치를 찾아 입력한다
        if str(lis[j].findAll('a')[0].find('span')) == 'None':
            a = 1
            while True:
                if str(lis[j].findAll('a')[a].find('span')) != 'None':
                    nicknames.append(lis[j].findAll('a')[a].find('span').getText())
                    break
                a += 1
        else:
            nicknames.append(lis[j].findAll('a')[0].find('span').getText())
        
        # 평점
        scores.append(lis[j].find('em').getText())
        
        # 댓글 내용
        review_text = lis[j].find('p').getText()
        if '관람객' in review_text:
            spectators.append(1) # 관람객인경우
        else:
            spectators.append(0) # 관람객이 아닐경우
        review_text = review_text.replace('관람객','')
        review_text = review_text.replace('\n','') # 줄바꿈표시 제거
        review_text = review_text.replace('\t','') # 탭 제거
        review_text = review_text.replace('\r','') # 맨앞표시 제거
        review_texts.append(review_text)
        
        # 좋아요/싫어요
        likes.append(lis[j].find('div', {'class': 'btn_area'}).findAll('strong')[0].getText())
        dislikes.append(lis[j].find('div', {'class': 'btn_area'}).findAll('strong')[1].getText())
        
        # 날짜
        date_list.append(lis[j].find('dt').findAll('em')[-1].getText())

100%|████████████████████████████████████████████████████████████████████████████████| 974/974 [01:48<00:00,  8.96it/s]


## DataFrame 생성

In [133]:
review_text

'그래서 교훈이 뭐지...?ㅠㅠ '

In [136]:
df = pd.DataFrame()
df['닉네임'] = nicknames
df['평점'] = scores
#df['관람여부'] = spectator
df['댓글내용'] = review_texts
df['좋아요'] = likes
df['싫어요'] = dislikes
df['날짜'] = date_list
df

,닉네임,평점,댓글내용,좋아요,싫어요,날짜
0,SIN(ultr****),10,당신이 무의미하다고 느끼는 일상의 나열같은 인생도 무의미하지 않다는걸 보여주는 연출...,3472,82,2021.01.20 13:41
1,복이와요(be1i****),10,"인생은 목적이 있어서, 이유가 있어서 사는게 아니라 사는 그 자체가 인생의 목적과 ...",2778,67,2021.01.20 16:27
2,kjc(jin3****),10,목적 없이 산다고 혼나는 줄 알았다.,1907,85,2021.01.20 12:59
3,즌므승(9mym****),10,진짜 내 인생 애니메이션으로 등극..,1248,58,2021.01.20 14:51
4,grac****,8,삶을 살아가면서 꼭 무엇을 이루기 위해 앞만 보고 치열하게 사는 삶에서 그저 하루하...,1027,40,2021.01.20 14:48
...,...,...,...,...,...,...
9730,이강철(kuma****),2,,0,11,2021.02.14 00:30
9731,안녕하세요(7316****),1,스포일러가 포함된 감상평입니다. 감상평 보기태어나지도 않은 영혼이고 모든것을 배워나...,1,14,2021.02.06 21:18
9732,kevj****,9,그저 그럭저럭 잘 보았읍니다. 딱히 남는것은 없읍니다,2,16,2021.04.11 08:40
9733,다크오크(hong****),8,너무 예쁜 여친하고 봐서 그런지 중간에 딴짓을....기대만큼은 아니지만 볼만해요,2,19,2021.01.24 22:02


In [141]:
df.to_csv("소울리뷰.csv", encoding='cp949', index=False)